In [1]:
# import libraries (non-cryptogaphic)
import random # to generate phone numbers
import pandas as pd


# import libraries (cryptographic)
import cryptography.hazmat.primitives.asymmetric.dh as dh
from cryptography.fernet import Fernet
import hashlib
import sympy
import secrets

# Generate phone numbers and store them in phone_numbers.csv file

In [2]:
random.seed(10) # to ensure same phone numbers generated every time

In [3]:
# class to generate phone numbers for grab and gojek
class PhoneNumberGenerator:
    def __call__(self, count):
        phone_numbers = random.sample(range(80000000,100000000), count)
        return phone_numbers 

    
# class to store numbers to csv
class PhoneNumberStorageManager:
    def __init__(self):
        self.filename = "phone_numbers.csv"
    def __call__(self, gojek_phone_numbers, grab_phone_numbers):
        d = {"gojek": gojek_phone_numbers, 
            "grab": grab_phone_numbers}
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])) # create dataframe
        df.to_csv(self.filename, index = False) # store values to file "phone_numbers.csv"

In [4]:
# generate and store phone numbers
gojek_phone_number_count = 61 # inclusive of phone numbers in common with grab
grab_phone_number_count = 91 # inclusive of phone numbers in common with gojek
common_phone_number_count = 10

# instantiate required classes
phone_number_generator = PhoneNumberGenerator()
phone_number_storage_manager = PhoneNumberStorageManager()

# generate phone numbers
phone_numbers = phone_number_generator(gojek_phone_number_count+grab_phone_number_count-common_phone_number_count)
common_phone_numbers = phone_numbers[0:common_phone_number_count]
gojek_phone_numbers = phone_numbers[0:gojek_phone_number_count]
grab_phone_numbers = common_phone_numbers + phone_numbers[gojek_phone_number_count:] 

#shuffle phone number lists
random.shuffle(gojek_phone_numbers)
random.shuffle(grab_phone_numbers)

# write phone numbers to csv file
phone_number_storage_manager(gojek_phone_numbers, grab_phone_numbers)

# Define classes for the necessary for the algorithm

In [5]:
# class to generate numbers required for psi
class NumberGenerator:
    
    def generate_public_parameters(self, size):
        # method to generate p, q and factors of p-1
        p = self.generate_safe_prime(size)
        print(f"p is prime: {sympy.ntheory.isprime(p)}")
        length_of_p = len(bin(p)[2:]) # should be 1024
        print(f"Length of prime modulus, p: {length_of_p}.\nNote: Should be {size}.")
        q = (p-1)//2
        print(f"q is prime: {sympy.ntheory.isprime(q)}")
        factors_pminus1 = [1, 2, q] # since q is prime, 2q only has these 3 factors excluding itself
                                    # 2q = p-1
                                    # factors of p-1 required to compute order of generators (lagrange theorem)
        return p, factors_pminus1

    def generate_safe_prime(self, size):
        # method to generate safe prime for p
        candidate = dh.generate_parameters(2, size).parameter_numbers().p # generate 1024-bit prime number
        while True:
            # the method used from the cryptography already generates a safe prime, this portion is merely to double confirm
            is_safe_prime = sympy.ntheory.isprime((candidate-1)//2) # if safe prime, (candidate-1)/2 is prime
            if (is_safe_prime):
                break
            else:
                candidate = dh.generate_parameters(2, size).parameter_numbers().p
                print(candidate)
 
        return candidate
    
    
    def generate_random_number(self, size):
        # method to generate client's secret
        return secrets.randbits(size)
            
# class to inspect values          
class NumberInspector:
    
    def check_is_primitive_generator(self, candidate, factors_divisorminus1, divisor): # note: factors should be the factors of divisor-1
        
        # apply lagrange theorem
        for possible_order in factors_divisorminus1: # check congruence for all factors (factors is exclusive of p-1 itself)
            result = pow(candidate, possible_order, divisor) # fast modular exponentiation implemented in pow(x,y,z)
                                                             # python integers have arbitrary precisions, 
                                                             # no overflow would occur if operations done in pure python
            if (result == 1):
                return False # if candidate exponent any of the factors (1,2,q) congruent to 1modp, candidate is not a primitive
                             # generator
        return True # order of candidate == p-1, therefore candidate is a primitive generator

    
class StorageManager:
    
    def store_data(self, filename, data):
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe
        df.to_csv(filename, index=False) # store data to file with filename

                


    
        

# Create psi client class

In [6]:
# client class (both grab and gojek are clients communicating directly with each other)
class Client:
    def __init__(self, name, other_party_name, private_key_size, phone_numbers, p, factors_pminus1, fernet_key):
        
        self.number_inspector = NumberInspector()
        self.number_generator = NumberGenerator()
        
        # for asymmetric encryption
        self.private_key = self.number_generator.generate_random_number(private_key_size)
        self.my_set = phone_numbers
        self.p = p # prime modulus
        self.factors_pminus1 = factors_pminus1 # to calculate order of hashed phone numbers, 
                                               # since algorithm requires them to be primitive generators
        
        # for symmetric encryption
        self.fernet_key = fernet_key
        self.f = Fernet(fernet_key)
        
        # values to track for psi algorithm
        self.my_hashed_set = None # h(x)
        self.my_self_encrypted_set = None # (h(x)^(my_secret))modp
        self.my_encrypted_set = None # (h(x)^(my_secret)(other_party_secret))modp
        self.other_party_encrypted_set = None # (h(y)^(my_secret)(other_party_secret))modp
        self.common_values = None # common phone numbers
        
        
        # create datafile for communication with another party
        
        # content to store in file
        # only need to share self_encrypted_values and other_party_encrypted_values
        # common_values to ensure both calculate the same intersection
        self.my_dict = {
            'my_self_encrypted_set': None, 
            'other_party_encrypted_set': None,
            'common_values': None
        }
        
        # filenames
        self.name = name
        self.filename = name + "_data_v1.1.csv"
        self.other_party_name = other_party_name
        self.other_party_filename = other_party_name + "_data_v1.1.csv"
        
        # create file
        self.storage_manager = StorageManager()
        self.storage_manager.store_data(self.filename, self.my_dict)

    def hash_to_primitive_root_modulo_p(self, element): 
        # method to hash phone numbers to primitive root modulo p i.e. primitive generator

        endian = "big"
        element = element.to_bytes(4, endian)
        hash_hex = hashlib.sha256(element).hexdigest() # sha3_256
        hash_int = int(hash_hex, 16)
        while True:
            # repeatedly hash until primitive root modulo p is obtained
            is_primitive_generator = self.number_inspector.check_is_primitive_generator(
                hash_int, self.factors_pminus1, self.p
            )
            if (is_primitive_generator):
                break
            else:
                hash_int = hash_int.to_bytes(32, endian)
                hash_hex = hashlib.sha256(hash_int).hexdigest()
                hash_int = int(hash_hex, 16)
                
        return hash_int
    
    def modular_exponentation(self, element):
        # compute (element^(private_key))modp

        return pow(element, self.private_key, self.p)
    
    def hash_set(self):
        # hash all phone numbers in my set to primitive root modulo p, one by one

        self.my_hashed_set = []
        
        for element in self.my_set:
            hashed_value = self.hash_to_primitive_root_modulo_p(element)
            self.my_hashed_set.append(hashed_value)
            
    def encrypt_set(self, is_other_party):
        # encrypt all elements in a given set using private_key, one by one
        
        # two scenarios to consider
        # one: encrypt set sent by the other party
        if (is_other_party):
            decrypted_other_party_set = self.receive_data("my_self_encrypted_set")
            other_party_set_int = []
            for element_string in decrypted_other_party_set:
                other_party_set_int.append(int(element_string))
            set_to_encrypt = other_party_set_int
        # two: encrypt my own set
        else:
            set_to_encrypt = self.my_hashed_set
        
        # encrypt values in given set, one by one
        encrypted_values = []
        for element in set_to_encrypt:
            encrypted_value = self.modular_exponentation(element)
            encrypted_values.append(encrypted_value)
            
        # assign the encrypted set to the correct variable
        # update csv file used for communication
        if (is_other_party):
            self.other_party_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "other_party_encrypted_set")
        else:
            self.my_self_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "my_self_encrypted_set")
                    
        
    def get_intersection(self):
        # get intersection
        
        my_encrypted_set = self.receive_data("other_party_encrypted_set") # read my encrypted set from the other party's file
        my_encrypted_set_int = []
        
        # convert read values to integer
        for element in my_encrypted_set:
            my_encrypted_set_int.append(int(element))
            
        # assign to correct variable
        self.my_encrypted_set = my_encrypted_set_int
        
        # get intersection
        encrypted_common_values = set(self.my_encrypted_set).intersection(self.other_party_encrypted_set)
        index_of_common_values = []
        
        # find the index of the elements in the intersection in my_encrypted_set
        for element in encrypted_common_values:
            index_of_common_values.append(self.my_encrypted_set.index(element))
            
        self.common_values = []
        
        # find the values in my own set corresponding to the index of the elements in the intersection
        for index in index_of_common_values:
            self.common_values.append(self.my_set[index])
            
        # update csv file for communication
        self.send_data(self.common_values, "common_values")
    
    def encrypt_data(self, plaintext):
        # encrypt data with Fernet
        
        endian = "big"
        element = plaintext.to_bytes(128, endian) # 1024 bits == 128 bytes
        cipher_text = self.f.encrypt(element)
        return cipher_text
    
    def decrypt_data(self, ciphertext):
        # decrypt data encrypted by Fernet
        
        ciphertext_bytes = ciphertext.encode('utf-8')[2:-1] # convert from string back to bytes
        endian = "big"
        element_in_bytes = self.f.decrypt(ciphertext_bytes)
        plaintext = int.from_bytes(element_in_bytes, endian)
        return plaintext

    def send_data(self, data_to_send, column_name):
        # send data means writing to file. encrypt data with Fernet
        
        # encrypt data
        encrypted_data_to_send = []
        for element in data_to_send:
            encrypted_element = self.encrypt_data(element)
            encrypted_data_to_send.append(encrypted_element)
            
        # send data
        self.my_dict[column_name] = encrypted_data_to_send
        self.storage_manager.store_data(self.filename, self.my_dict)
        
        
    def receive_data(self, column_name):
        # receive data means reading from file (my file). decrypt data encrypted by Fernet
        
        # receive data
        encrypted_data = self.get_other_party_data()[column_name].to_list()
        
        # decrypt data
        decrypted_data = []
        for element in encrypted_data:
            if type(element) is float: # remove NaN
                continue
            decrypted_element = self.decrypt_data(element)
            decrypted_data.append(decrypted_element)
        print(f"number of elements in {self.other_party_name};{column_name}: {len(decrypted_data)}.")
        return decrypted_data
    
    def get_my_data(self):
        # read my file as dataframe (other party's file)
        
        return pd.read_csv(self.filename)
            
    def get_other_party_data(self):
        # read other party's file as dataframe
        
        return pd.read_csv(self.other_party_filename)
    


# Initialize context

In [7]:
# assign pre-determined variables for psi
key_size = 1024 # both private keys and large prime

# create key for symmetric key cryptography
fernet_key = Fernet.generate_key()

# create public parameters required for psi
number_generator = NumberGenerator()
p, factors_pminus1 = number_generator.generate_public_parameters(key_size)

# create clients
grab = Client("grab", "gojek", key_size, grab_phone_numbers, p, factors_pminus1, fernet_key)
gojek = Client("gojek", "grab", key_size, gojek_phone_numbers, p, factors_pminus1, fernet_key)



p is prime: True
Length of prime modulus, p: 1024.
Note: Should be 1024.
q is prime: True


C:\Users\jiaji\AppData\Local\Temp/ipykernel_6572/964370003.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


# Get intersection

## Step 1: Hash phone numbers

In [8]:
# clients hash their own set
grab.hash_set()
gojek.hash_set()

### Clients' status after step 1 (value of variables in client)
Note: hashed set is not stored in the file as it is not meant to be shared with the other party hence, files' status after step 1 is not shown

In [9]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set
0,1061642916839729595311745988861392474934466737...,1751659343934249496930093781941606327015039027...
1,8697318885625964372170362596957150767783631535...,2369003795800176044803624843724327263177283800...
2,6456970480652929440281673297917938501061617606...,3003360056476023280396172869223514448918734055...
3,4271190063387899149690355220761209149002412790...,5550251095231114107536200284368885541285504508...
4,1959933681547377774404303441869569823311439691...,5420082799610814861471144105653222128483755681...
...,...,...
86,NaN,3922283845005472934707328127251028806214862373...
87,NaN,3683371775181761031500661295222456997304332645...
88,NaN,1072449155912549032182436416689256902498507221...
89,NaN,6836305502874994557659887093477828849206228041...


## Step 2: encrypt hashed set with own private key

In [10]:
# clients self encrypt hashed set
grab.encrypt_set(False) # set is_other_party to false to encrypt own hashed set
gojek.encrypt_set(False)

C:\Users\jiaji\AppData\Local\Temp/ipykernel_6572/964370003.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


### Clients' status after step 2 (value of variables in client)

In [11]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set
0,1061642916839729595311745988861392474934466737...,1751659343934249496930093781941606327015039027...,6287284300867062990658787904018295880428913273...,9700401470038044779439023350214083024249639961...
1,8697318885625964372170362596957150767783631535...,2369003795800176044803624843724327263177283800...,1561712674404630506810920949807849312797368598...,8294845719441555848895090999279652837077414800...
2,6456970480652929440281673297917938501061617606...,3003360056476023280396172869223514448918734055...,1561557439189394368794346814859451299200167570...,8440187735061807478535617937493148397139614944...
3,4271190063387899149690355220761209149002412790...,5550251095231114107536200284368885541285504508...,2850881475948583578779462632695568770693467687...,4109298350872153662188147229322151869168539287...
4,1959933681547377774404303441869569823311439691...,5420082799610814861471144105653222128483755681...,1607042718424108617145202669799199413290587008...,1168197542691792925496275447931556919092062557...
...,...,...,...,...
86,NaN,3922283845005472934707328127251028806214862373...,NaN,9258931915062586261466149961918648670699530906...
87,NaN,3683371775181761031500661295222456997304332645...,NaN,1092470063382416728433378489366973841574762812...
88,NaN,1072449155912549032182436416689256902498507221...,NaN,8058971515999780816309873081490403620119997086...
89,NaN,6836305502874994557659887093477828849206228041...,NaN,8399510593129024758085448461740375496648317722...


### Files' status after step 2 (value of variables in file - clients' variables encrypted with Fernet)

In [12]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhofNRGD1tyAgCL6NvWpLyWNoZxnghTPbrqNTu...,NaN,NaN
1,b'gAAAAABhofNR0DthXl3VIXbO_o7NZZhgn7eNlJziMyIq...,NaN,NaN
2,b'gAAAAABhofNRcowI25GV_VY84zN3RC32F81ddX19YJO6...,NaN,NaN
3,b'gAAAAABhofNRQv3TEPwEHI6pzQIWGG4Q3w010rlCN7nq...,NaN,NaN
4,b'gAAAAABhofNRziw1JtJtIRgHAytL0-Xv_S3rxBoW5SG1...,NaN,NaN
...,...,...,...
56,b'gAAAAABhofNRISrmDzSMX_41A9L4BpO0wyh5sTfdM9FQ...,NaN,NaN
57,b'gAAAAABhofNRgeU1XA2T7k8YNl8mYKc-shmfm1op8uUK...,NaN,NaN
58,b'gAAAAABhofNRDRA6iL1k_1RZorjP1tcBzm3qHjdYs14p...,NaN,NaN
59,b'gAAAAABhofNR3nbVi3IIbWopB7QH1Dwoa9Op9u9cMWJv...,NaN,NaN


In [13]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhofNR42OL9y-EJBZc0bTv9c70zvEtruwbNRNh...,NaN,NaN
1,b'gAAAAABhofNRXz4ry_RXmZaOrPAXzyIKPGaXgS-ykF5u...,NaN,NaN
2,b'gAAAAABhofNRrEVvb2b6CWbDnOXKyPm3RxYXnieMHBOm...,NaN,NaN
3,b'gAAAAABhofNRAsPkhKT_k6RKOUYCRp3ymQmByymEnnCu...,NaN,NaN
4,b'gAAAAABhofNRNHYQOr24ifogU94ULoJMilDD-LHlJo2t...,NaN,NaN
...,...,...,...
86,b'gAAAAABhofNRYI09SzOzyCxIPKbGWC4i_J_ypFbJ0JDs...,NaN,NaN
87,b'gAAAAABhofNR8YcmcT8mpntTnwbK7sztF-MQFdvShP3r...,NaN,NaN
88,b'gAAAAABhofNRHksPdC6DfeEI5qokk2W_91ZhAXa2oeby...,NaN,NaN
89,b'gAAAAABhofNRP_mkfJmgfKD-vVAJYBJ6FBWSk8K8xrOK...,NaN,NaN


## Step 3: encrypt other party's self-encrypted set with own private key


In [14]:
# clients encrypt other party's self encrypted set
grab.encrypt_set(True) # set is_other_party to true
gojek.encrypt_set(True)

number of elements in gojek;my_self_encrypted_set: 61.


C:\Users\jiaji\AppData\Local\Temp/ipykernel_6572/964370003.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


number of elements in grab;my_self_encrypted_set: 91.


### Clients' status after step 3

In [15]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set
0,1061642916839729595311745988861392474934466737...,1751659343934249496930093781941606327015039027...,6287284300867062990658787904018295880428913273...,9700401470038044779439023350214083024249639961...,9747652482325446472978084658738961292568302192...,1243084293356201123091647406710010987346469756...
1,8697318885625964372170362596957150767783631535...,2369003795800176044803624843724327263177283800...,1561712674404630506810920949807849312797368598...,8294845719441555848895090999279652837077414800...,1409978992958062003965011810207735034591369082...,1076245430793902850238123588877280958459010705...
2,6456970480652929440281673297917938501061617606...,3003360056476023280396172869223514448918734055...,1561557439189394368794346814859451299200167570...,8440187735061807478535617937493148397139614944...,1101395179620158772273356516022419739410882106...,3074445920394552760782872527982571725031415672...
3,4271190063387899149690355220761209149002412790...,5550251095231114107536200284368885541285504508...,2850881475948583578779462632695568770693467687...,4109298350872153662188147229322151869168539287...,1250455102031725960416862854729353365794861918...,5215937215373710954220742324760892911422382626...
4,1959933681547377774404303441869569823311439691...,5420082799610814861471144105653222128483755681...,1607042718424108617145202669799199413290587008...,1168197542691792925496275447931556919092062557...,1339833090404037956536610136249014151532204723...,1187957741681301904548827192744360779086031911...
...,...,...,...,...,...,...
86,NaN,3922283845005472934707328127251028806214862373...,NaN,9258931915062586261466149961918648670699530906...,NaN,3749038566268576675447767781216931070489142299...
87,NaN,3683371775181761031500661295222456997304332645...,NaN,1092470063382416728433378489366973841574762812...,NaN,1151905057308242506174989173859318357846875555...
88,NaN,1072449155912549032182436416689256902498507221...,NaN,8058971515999780816309873081490403620119997086...,NaN,2246471895987317734664360262944303440168551396...
89,NaN,6836305502874994557659887093477828849206228041...,NaN,8399510593129024758085448461740375496648317722...,NaN,1414177516080253386756729498891116012208653720...


### Files' status after step 3

In [16]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek



gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhofNRGD1tyAgCL6NvWpLyWNoZxnghTPbrqNTu...,b'gAAAAABhofNSue89fFpgeabMQOgaLu-2mInuQxLy70Fk...,NaN
1,b'gAAAAABhofNR0DthXl3VIXbO_o7NZZhgn7eNlJziMyIq...,b'gAAAAABhofNSLhtTnolydQRm6EmhC7Xk9QWPw1-z4fst...,NaN
2,b'gAAAAABhofNRcowI25GV_VY84zN3RC32F81ddX19YJO6...,b'gAAAAABhofNSiTQEq1KYr2tg7d9p7HHkKuFQHpBAPBEV...,NaN
3,b'gAAAAABhofNRQv3TEPwEHI6pzQIWGG4Q3w010rlCN7nq...,b'gAAAAABhofNS5ejtn2LryiLAKq4O0b1HfyMeljmMbrA4...,NaN
4,b'gAAAAABhofNRziw1JtJtIRgHAytL0-Xv_S3rxBoW5SG1...,b'gAAAAABhofNSvyr0YtNwYi5IATbAwlpqvumwK_Jj9TW3...,NaN
...,...,...,...
86,NaN,b'gAAAAABhofNSgqLoDatb3fdAsuGeGXirvqmS_oQu7V50...,NaN
87,NaN,b'gAAAAABhofNSRZhWNS9HC8KUA6LdMrW2Oh4BQrZtbb4B...,NaN
88,NaN,b'gAAAAABhofNSHMrJZNa8EqQk5ac-gZ9pFQsrvWKGdW0x...,NaN
89,NaN,b'gAAAAABhofNSJHmNF4jeHib_Bftm0VJcGS4ATRzZq_hU...,NaN


In [17]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhofNR42OL9y-EJBZc0bTv9c70zvEtruwbNRNh...,b'gAAAAABhofNSYdHoQMGNumscZzLUJSnNVXknbkUmL5Lp...,NaN
1,b'gAAAAABhofNRXz4ry_RXmZaOrPAXzyIKPGaXgS-ykF5u...,b'gAAAAABhofNSk8IFnNS8oDt9KN_oxKErOfo7eQ-2lqVT...,NaN
2,b'gAAAAABhofNRrEVvb2b6CWbDnOXKyPm3RxYXnieMHBOm...,b'gAAAAABhofNSWxQh_ZY3jaYZqtJpfw0vF5rvFKt6-_9Y...,NaN
3,b'gAAAAABhofNRAsPkhKT_k6RKOUYCRp3ymQmByymEnnCu...,b'gAAAAABhofNSsQ7B7QdH8yxZfxjbwQ6nf3CxqrYIDP7Q...,NaN
4,b'gAAAAABhofNRNHYQOr24ifogU94ULoJMilDD-LHlJo2t...,b'gAAAAABhofNSJVHtSjVRrpG5p_jZSLiRZL0N2i45esX8...,NaN
...,...,...,...
86,b'gAAAAABhofNRYI09SzOzyCxIPKbGWC4i_J_ypFbJ0JDs...,NaN,NaN
87,b'gAAAAABhofNR8YcmcT8mpntTnwbK7sztF-MQFdvShP3r...,NaN,NaN
88,b'gAAAAABhofNRHksPdC6DfeEI5qokk2W_91ZhAXa2oeby...,NaN,NaN
89,b'gAAAAABhofNRP_mkfJmgfKD-vVAJYBJ6FBWSk8K8xrOK...,NaN,NaN


## Step 4: find intersection


In [18]:
# clients find intersection
grab.get_intersection()
gojek.get_intersection()

number of elements in gojek;other_party_encrypted_set: 91.
number of elements in grab;other_party_encrypted_set: 61.


### Clients' status after step 4:

In [19]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set,
    "gojek found intersection": gojek.common_values,
    "grab found intersection:": grab.common_values}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set,gojek found intersection,grab found intersection:
0,1061642916839729595311745988861392474934466737...,1751659343934249496930093781941606327015039027...,6287284300867062990658787904018295880428913273...,9700401470038044779439023350214083024249639961...,9747652482325446472978084658738961292568302192...,1243084293356201123091647406710010987346469756...,86915509.0,86915509.0
1,8697318885625964372170362596957150767783631535...,2369003795800176044803624843724327263177283800...,1561712674404630506810920949807849312797368598...,8294845719441555848895090999279652837077414800...,1409978992958062003965011810207735034591369082...,1076245430793902850238123588877280958459010705...,95521626.0,95521626.0
2,6456970480652929440281673297917938501061617606...,3003360056476023280396172869223514448918734055...,1561557439189394368794346814859451299200167570...,8440187735061807478535617937493148397139614944...,1101395179620158772273356516022419739410882106...,3074445920394552760782872527982571725031415672...,89312048.0,89312048.0
3,4271190063387899149690355220761209149002412790...,5550251095231114107536200284368885541285504508...,2850881475948583578779462632695568770693467687...,4109298350872153662188147229322151869168539287...,1250455102031725960416862854729353365794861918...,5215937215373710954220742324760892911422382626...,99173089.0,94391128.0
4,1959933681547377774404303441869569823311439691...,5420082799610814861471144105653222128483755681...,1607042718424108617145202669799199413290587008...,1168197542691792925496275447931556919092062557...,1339833090404037956536610136249014151532204723...,1187957741681301904548827192744360779086031911...,81093373.0,99173089.0
...,...,...,...,...,...,...,...,...
86,NaN,3922283845005472934707328127251028806214862373...,NaN,9258931915062586261466149961918648670699530906...,NaN,3749038566268576675447767781216931070489142299...,NaN,NaN
87,NaN,3683371775181761031500661295222456997304332645...,NaN,1092470063382416728433378489366973841574762812...,NaN,1151905057308242506174989173859318357846875555...,NaN,NaN
88,NaN,1072449155912549032182436416689256902498507221...,NaN,8058971515999780816309873081490403620119997086...,NaN,2246471895987317734664360262944303440168551396...,NaN,NaN
89,NaN,6836305502874994557659887093477828849206228041...,NaN,8399510593129024758085448461740375496648317722...,NaN,1414177516080253386756729498891116012208653720...,NaN,NaN


Note: Last 2 columns, unlike the rest of the columns, do not have a one-to-one mapping with other values belonging to the same row i.e. values in the last 2 columns do not have any relation to the other values in the same row as it.

### Files' status after step 4

In [20]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhofNRGD1tyAgCL6NvWpLyWNoZxnghTPbrqNTu...,b'gAAAAABhofNSue89fFpgeabMQOgaLu-2mInuQxLy70Fk...,b'gAAAAABhofNS_Frpwrt4uuutun1HBbSWNlFb-NyuZu37...
1,b'gAAAAABhofNR0DthXl3VIXbO_o7NZZhgn7eNlJziMyIq...,b'gAAAAABhofNSLhtTnolydQRm6EmhC7Xk9QWPw1-z4fst...,b'gAAAAABhofNS4WqXLO8Cx6DFfYDDAuENb2Qs7Opvid9X...
2,b'gAAAAABhofNRcowI25GV_VY84zN3RC32F81ddX19YJO6...,b'gAAAAABhofNSiTQEq1KYr2tg7d9p7HHkKuFQHpBAPBEV...,b'gAAAAABhofNSs9qXx2_RznPnMOqTD2ErcPMl-UrBVXkj...
3,b'gAAAAABhofNRQv3TEPwEHI6pzQIWGG4Q3w010rlCN7nq...,b'gAAAAABhofNS5ejtn2LryiLAKq4O0b1HfyMeljmMbrA4...,b'gAAAAABhofNSxGNBHvdDU1R2KGx6vCtbVEZo2GhlQcfh...
4,b'gAAAAABhofNRziw1JtJtIRgHAytL0-Xv_S3rxBoW5SG1...,b'gAAAAABhofNSvyr0YtNwYi5IATbAwlpqvumwK_Jj9TW3...,b'gAAAAABhofNS9bPoEMTfh162INUIclFfSqp8OM3MZzpA...
...,...,...,...
86,NaN,b'gAAAAABhofNSgqLoDatb3fdAsuGeGXirvqmS_oQu7V50...,NaN
87,NaN,b'gAAAAABhofNSRZhWNS9HC8KUA6LdMrW2Oh4BQrZtbb4B...,NaN
88,NaN,b'gAAAAABhofNSHMrJZNa8EqQk5ac-gZ9pFQsrvWKGdW0x...,NaN
89,NaN,b'gAAAAABhofNSJHmNF4jeHib_Bftm0VJcGS4ATRzZq_hU...,NaN


In [21]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhofNR42OL9y-EJBZc0bTv9c70zvEtruwbNRNh...,b'gAAAAABhofNSYdHoQMGNumscZzLUJSnNVXknbkUmL5Lp...,b'gAAAAABhofNSa7BDkRK4-XWakp_tPrjbLc3nTJAaIw5s...
1,b'gAAAAABhofNRXz4ry_RXmZaOrPAXzyIKPGaXgS-ykF5u...,b'gAAAAABhofNSk8IFnNS8oDt9KN_oxKErOfo7eQ-2lqVT...,b'gAAAAABhofNSffN5NI9SjG7WpnbpfQq7vEAy37TnTvwZ...
2,b'gAAAAABhofNRrEVvb2b6CWbDnOXKyPm3RxYXnieMHBOm...,b'gAAAAABhofNSWxQh_ZY3jaYZqtJpfw0vF5rvFKt6-_9Y...,b'gAAAAABhofNSpfX9GeSvWDyULhuiEKTkCFLes5sMjPCp...
3,b'gAAAAABhofNRAsPkhKT_k6RKOUYCRp3ymQmByymEnnCu...,b'gAAAAABhofNSsQ7B7QdH8yxZfxjbwQ6nf3CxqrYIDP7Q...,b'gAAAAABhofNSabLlhy19E8XejK488pthlVJD7cTBY9QR...
4,b'gAAAAABhofNRNHYQOr24ifogU94ULoJMilDD-LHlJo2t...,b'gAAAAABhofNSJVHtSjVRrpG5p_jZSLiRZL0N2i45esX8...,b'gAAAAABhofNSVzSkf2zEAjxJo_lsFJNgTwZIgPVWiVPj...
...,...,...,...
86,b'gAAAAABhofNRYI09SzOzyCxIPKbGWC4i_J_ypFbJ0JDs...,NaN,NaN
87,b'gAAAAABhofNR8YcmcT8mpntTnwbK7sztF-MQFdvShP3r...,NaN,NaN
88,b'gAAAAABhofNRHksPdC6DfeEI5qokk2W_91ZhAXa2oeby...,NaN,NaN
89,b'gAAAAABhofNRP_mkfJmgfKD-vVAJYBJ6FBWSk8K8xrOK...,NaN,NaN


# Check results

In [22]:
# get intersection found by the two parties
gojek_found_intersection = gojek.common_values
grab_found_intersection = grab.common_values


# sort numbers for easier comparison
gojek_found_intersection.sort()
grab_found_intersection.sort()
common_phone_numbers.sort()

# summarize them in a dataframe
d = {"actual": common_phone_numbers,
    "gojek": gojek_found_intersection,
    "grab": grab_found_intersection}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,actual,gojek,grab
0,80497694,80497694,80497694
1,81093373,81093373,81093373
2,86915509,86915509,86915509
3,89312048,89312048,89312048
4,94391128,94391128,94391128
5,95521626,95521626,95521626
6,96192082,96192082,96192082
7,96485172,96485172,96485172
8,99173089,99173089,99173089
9,99397525,99397525,99397525
